In [2]:
import chess.pgn
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [3]:
def board_to_matrix(board: chess.Board, move: chess.Move):
    matrix_board = torch.zeros((6, 8, 8))
    matrix_move = torch.zeros((8, 8))

    for i in range(8):
        for j in range(8):
            piece = board.piece_at(chess.square(i, j))
            if piece is not None:
                piece_type = piece.piece_type
                piece_color = piece.color
                index = piece_type - 1
                if piece_color == chess.WHITE:
                    matrix_board[index, 7-j, i] = 1
                else:
                    matrix_board[index, 7-j, i] = -1
                # print(matrix_board)
    # if board.turn == chess.BLACK:
    #     matrix_board *= -1
    #     matrix_board = [torch.flip(matrix, dims=[0]) for matrix in matrix_board]

    file = chess.square_file(move.from_square)
    rank = chess.square_rank(move.from_square)
    matrix_move[7-rank][file] = 1
    # print(matrix_board)
    return matrix_board.tolist(), matrix_move.tolist()

def main():
    pgn = open("full.pgn")
    cnt = 0
    board_matrix, piece_matrix = [], []

    while True:
        game = chess.pgn.read_game(pgn)
        if game is None:
            break
        cnt+=1
        if cnt % 500 == 0:
            print(cnt)

        board = game.board()
        for move in game.mainline_moves():
          if board.turn == chess.WHITE:
              matrix_board, matrix_move = board_to_matrix(board, move)
              board_matrix.append(matrix_board)
              piece_matrix.append(matrix_move)
          board.push(move)


    X = torch.tensor(board_matrix)
    y = torch.tensor(piece_matrix)
    torch.save(X, "X.pt")
    torch.save(y, "y.pt")

main()

500
1000
1500
2000
2500
3000
3500
4000
4500
5000


In [10]:
def test(device_str: str, eps_num: int, leraning_rate = 0.03):
  X = torch.load("X.pt")
  y = torch.load("y.pt")
  assert len(X) == len(y)

  ratio = 0.8
  idx = int(X.size(0)*ratio)
  device = torch.device(device_str if torch.cuda.is_available() else "cpu")
  if device_str == "cuda":
    X_train, X_test = X[:idx].to(device), X[idx:].to(device)
    Y_train, Y_test = y[:idx].to(device), y[idx:].to(device)
  else:
    X_train, X_test = X[:idx], X[idx:]
    Y_train, Y_test = y[:idx], y[idx:]
  print(torch.cuda.is_available())
  print(device)

  model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(8*8*6, 64),
    nn.ReLU(),
    nn.Linear(64, 64),
    nn.ReLU(),
    nn.Linear(64, 64),
    nn.ReLU(),
    nn.Linear(64, 64),
    nn.ReLU(),
    nn.Linear(64, 64),
    nn.ReLU(),
    nn.Linear(64, 64),
    nn.ReLU(),
    nn.Linear(64, 8*8),
  ).to(device)

  X_train.to(device)
  X_test.to(device)
  Y_train.to(device)
  Y_test.to(device)

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(model.parameters(), lr=leraning_rate)
  num_epochs = eps_num
  eps = []
  train_acc = []
  test_acc = []

  for epoch in range(num_epochs):
    optimizer.zero_grad()
    output = model(X_train.view(-1, 8*8*6))
    loss = criterion(output, Y_train.view(-1, 8*8))
    loss.backward()
    optimizer.step()

    if (epoch+1) % 3000 == 0:
      model.eval()
      correct = 0
      total = 0
      with torch.no_grad():
        val_inputs = X_test.view(-1, 8*8*6)
        val_labels = Y_test.view(-1, 8*8)
        val_outputs = model(val_inputs)
        val_loss = criterion(val_outputs, val_labels)
        max_idxs, predicted = torch.max(val_outputs, 1)
        total = val_labels.size(0)
        correct = torch.sum((val_labels[torch.arange(len(predicted)), predicted] == 1).float())
        accuracy_test = correct / total

        val_inputs = X_train.view(-1, 8*8*6)
        val_labels = Y_train.view(-1, 8*8)
        val_outputs = model(val_inputs)
        val_loss = criterion(val_outputs, val_labels)
        _, predicted = torch.max(val_outputs, 1)
        total = val_labels.size(0)
        correct = torch.sum((val_labels[torch.arange(len(predicted)), predicted] == 1).float())
        accuracy_train = correct / total

        print(f'Epoch [{epoch+1}/{num_epochs}], train_acc: {accuracy_train:4f}, test_acc: {accuracy_test:4f}')
        eps.append(epoch)
        train_acc.append(accuracy_train.cpu().numpy())  # Konwersja tensora na numpy array
        test_acc.append(accuracy_test.cpu().numpy()) 

  plt.plot(eps, train_acc, label="train_acc")
  plt.plot(eps, test_acc, label="test_acc")
  plt.xlabel("eps")
  plt.ylabel("accuracy")
  plt.legend()
  plt.title(f"Accuracy, lr: {leraning_rate}")
  plt.show()


if __name__ == "__main__":
  # current_time = datetime.datetime.now()
  eps = 300000
  test("cuda", eps)
  # end = datetime.datetime.now()
  # print(f"cuda {eps} eps: {end-current_time}")
  # current_time = datetime.datetime.now()
  # test("cpu", 100)
  # end = datetime.datetime.now()
  # print(f"cuda 100 eps: {end-current_time}")



True
cuda
Epoch [3000/300000], train_acc: 0.056809, test_acc: 0.055678
Epoch [6000/300000], train_acc: 0.070778, test_acc: 0.070597
Epoch [9000/300000], train_acc: 0.072766, test_acc: 0.073677
Epoch [12000/300000], train_acc: 0.083726, test_acc: 0.082698
Epoch [15000/300000], train_acc: 0.113024, test_acc: 0.109609
Epoch [18000/300000], train_acc: 0.128385, test_acc: 0.123064
Epoch [21000/300000], train_acc: 0.137690, test_acc: 0.128853
Epoch [24000/300000], train_acc: 0.140497, test_acc: 0.129617
Epoch [27000/300000], train_acc: 0.163296, test_acc: 0.145956
Epoch [30000/300000], train_acc: 0.176888, test_acc: 0.157139
Epoch [33000/300000], train_acc: 0.183917, test_acc: 0.162731
Epoch [36000/300000], train_acc: 0.188733, test_acc: 0.164785
Epoch [39000/300000], train_acc: 0.194336, test_acc: 0.168367
Epoch [42000/300000], train_acc: 0.221094, test_acc: 0.191564
Epoch [45000/300000], train_acc: 0.230847, test_acc: 0.199122
Epoch [48000/300000], train_acc: 0.240153, test_acc: 0.206134
E